In [113]:
import numpy as np 
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer # преобразовывает названия городов в m биномиальных признаков
import scipy
from sklearn.linear_model import Ridge

In [114]:
train = pd.read_csv('salary-train.csv')
test = pd.read_csv('salary-test-mini.csv')

## Подготовка текста

Первым делом всё приводится к нижнему регистру, все символы кроме букв и цифр заменяются на пробелы. Далее мы работаем с классификационными признаками (например: город, название фирмы), мы должны преобразовать название в соответствующий номер. После этого обучаем регрессию с L2 регуляризацией

In [115]:
train['FullDescription'] = [train['FullDescription'][i].lower() for i in range(train['FullDescription'].shape[0])]

In [116]:
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [117]:
tdf = TfidfVectorizer(min_df = 5)

In [118]:
train_desc = tdf.fit_transform(train['FullDescription'])
test_desc = tdf.transform(test['FullDescription'])

In [119]:
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

In [120]:
enc = DictVectorizer()

In [121]:
X_train_categ = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [122]:
train_data = scipy.sparse.hstack([X_train_categ, train_desc])
test_data = scipy.sparse.hstack([X_test_categ, test_desc])

In [123]:
y_train = train['SalaryNormalized']

In [124]:
rg = Ridge(alpha = 1,random_state=241)

In [125]:
rg.fit(train_data,y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=241, solver='auto', tol=0.001)

In [126]:
rg.predict(test_data)

array([56555.61500155, 37188.32442618])